## Geopandas Functions to Calculate Green Space Percentage

In [1]:
import pandas as pd
import numpy as ny
import geopandas as gpd

C:\Users\teres\AppData\Local\Temp\ipykernel_3096\3568268968.py:3: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [128]:
#set radius of buffer in here
RADIUS_BUFFER = 500

In [141]:
#Function to create centriod and buffer 

def centriod_buffer(df):
    df["centriod"]=df.geometry.centroid
    df["buffer"]=df.geometry.buffer(RADIUS_BUFFER)

In [142]:
#function to calculate green space ratio

def green_percentage(df, numerator_col, denominator_col="grid_area"):
    def get_buffer_percentage(buffer):
        intersects = df["geometry"].intersects(buffer)
        denominator = df[intersects][denominator_col].sum()
        numerator = df[intersects][numerator_col].sum()
        return numerator/denominator*100

    buffer_series = df["buffer"]
    return buffer_series.map(get_buffer_percentage)


### Import Designated Green Space GeoJson

In [138]:
#designated_greenspace

dgs=gpd.read_file("data/geojson/designated_greenspace.geojson")
dgs.head(1)

,id,left,top,right,bottom,grid_area,total_gp_area,tgp_area%,dgs_area,dgs%,qusai_area,qgs%,water_area,water%,wild_area,wild%,geometry
0,205,627826.686572,6.932210e+06,627926.686572,6.932110e+06,146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627911.921 6932119.478, 627926..."


In [ ]:
#Change data type to float

dgs=dgs.astype({
    'grid_area':'float',
    'total_gp_area':'float',
    'tgp_area%':'float',
    'dgs_area':'float',
    'dgs%':'float',
    'qusai_area':'float',
    'qgs%':'float',
    'water_area':'float',
    'water%':'float',
    'wild_area':'float',
    'wild%':'float'
})

In [144]:
centriod_buffer(dgs)
dgs.head()

,id,left,top,right,bottom,grid_area,total_gp_area,tgp_area%,dgs_area,dgs%,qusai_area,qgs%,water_area,water%,wild_area,wild%,geometry,centriod,buffer
0,205,627826.686572,6.932210e+06,627926.686572,6.932110e+06,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627911.921 6932119.478, 627926...",POINT (627919.464 6932114.783),"POLYGON ((627412.278 6932100.593, 627412.832 6..."
1,206,627826.686572,6.932110e+06,627926.686572,6.932010e+06,1251.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627912.289 6932109.758, 627926...",POINT (627920.385 6932062.276),"POLYGON ((627412.646 6932090.873, 627413.273 6..."
2,207,627826.686572,6.932010e+06,627926.686572,6.931910e+06,873.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627916.069 6932009.758, 627926...",POINT (627922.254 6931963.367),"POLYGON ((627416.425 6931990.873, 627417.053 6..."
3,208,627826.686572,6.931910e+06,627926.686572,6.931810e+06,495.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627919.848 6931909.758, 627926...",POINT (627924.092 6931866.123),"POLYGON ((627420.205 6931890.873, 627420.832 6..."
4,209,627826.686572,6.931810e+06,627926.686572,6.931710e+06,124.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627923.628 6931809.758, 627926...",POINT (627925.667 6931782.783),"POLYGON ((627423.985 6931790.873, 627424.612 6..."


In [145]:
dgs["green_percentage"] = green_percentage(dgs, "dgs_area")
dgs.head()

,id,left,top,right,bottom,grid_area,total_gp_area,tgp_area%,dgs_area,dgs%,qusai_area,qgs%,water_area,water%,wild_area,wild%,geometry,centriod,buffer,green_percentage
0,205,627826.686572,6.932210e+06,627926.686572,6.932110e+06,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627911.921 6932119.478, 627926...",POINT (627919.464 6932114.783),"POLYGON ((627412.278 6932100.593, 627412.832 6...",0.0
1,206,627826.686572,6.932110e+06,627926.686572,6.932010e+06,1251.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627912.289 6932109.758, 627926...",POINT (627920.385 6932062.276),"POLYGON ((627412.646 6932090.873, 627413.273 6...",0.0
2,207,627826.686572,6.932010e+06,627926.686572,6.931910e+06,873.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627916.069 6932009.758, 627926...",POINT (627922.254 6931963.367),"POLYGON ((627416.425 6931990.873, 627417.053 6...",0.0
3,208,627826.686572,6.931910e+06,627926.686572,6.931810e+06,495.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627919.848 6931909.758, 627926...",POINT (627924.092 6931866.123),"POLYGON ((627420.205 6931890.873, 627420.832 6...",0.0
4,209,627826.686572,6.931810e+06,627926.686572,6.931710e+06,124.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627923.628 6931809.758, 627926...",POINT (627925.667 6931782.783),"POLYGON ((627423.985 6931790.873, 627424.612 6...",0.0


### Import Qusai Green Space GeoJson

In [82]:
#quasi_greenspace

qgs=gpd.read_file("data/geojson/quasi_greenspace.geojson")

In [147]:
#Change data type to float

qgs=qgs.astype({
    'grid_area':'float',
    'total_gp_area':'float',
    'tgp_area%':'float',
    'dgs_area':'float',
    'dgs%':'float',
    'qusai_area':'float',
    'qgs%':'float',
    'water_area':'float',
    'water%':'float',
    'wild_area':'float',
    'wild%':'float'
})

In [148]:
centriod_buffer(qgs)
qgs.head()

,id,left,top,right,bottom,grid_area,total_gp_area,tgp_area%,dgs_area,dgs%,qusai_area,qgs%,water_area,water%,wild_area,wild%,geometry,centriod,buffer
0,205,627826.686572,6.932210e+06,627926.686572,6.932110e+06,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627911.921 6932119.478, 627926...",POINT (627919.464 6932114.783),"POLYGON ((627412.278 6932100.593, 627412.832 6..."
1,206,627826.686572,6.932110e+06,627926.686572,6.932010e+06,1251.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627912.289 6932109.758, 627926...",POINT (627920.385 6932062.276),"POLYGON ((627412.646 6932090.873, 627413.273 6..."
2,207,627826.686572,6.932010e+06,627926.686572,6.931910e+06,873.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627916.069 6932009.758, 627926...",POINT (627922.254 6931963.367),"POLYGON ((627416.425 6931990.873, 627417.053 6..."
3,208,627826.686572,6.931910e+06,627926.686572,6.931810e+06,495.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627919.848 6931909.758, 627926...",POINT (627924.092 6931866.123),"POLYGON ((627420.205 6931890.873, 627420.832 6..."
4,209,627826.686572,6.931810e+06,627926.686572,6.931710e+06,124.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627923.628 6931809.758, 627926...",POINT (627925.667 6931782.783),"POLYGON ((627423.985 6931790.873, 627424.612 6..."


In [152]:
qgs["green_percentage"] = green_percentage(qgs, "qusai_area")
qgs.head()

,id,left,top,right,bottom,grid_area,total_gp_area,tgp_area%,dgs_area,dgs%,qusai_area,qgs%,water_area,water%,wild_area,wild%,geometry,centriod,buffer,green_percentage
0,205,627826.686572,6.932210e+06,627926.686572,6.932110e+06,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627911.921 6932119.478, 627926...",POINT (627919.464 6932114.783),"POLYGON ((627412.278 6932100.593, 627412.832 6...",0.0
1,206,627826.686572,6.932110e+06,627926.686572,6.932010e+06,1251.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627912.289 6932109.758, 627926...",POINT (627920.385 6932062.276),"POLYGON ((627412.646 6932090.873, 627413.273 6...",0.0
2,207,627826.686572,6.932010e+06,627926.686572,6.931910e+06,873.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627916.069 6932009.758, 627926...",POINT (627922.254 6931963.367),"POLYGON ((627416.425 6931990.873, 627417.053 6...",0.0
3,208,627826.686572,6.931910e+06,627926.686572,6.931810e+06,495.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627919.848 6931909.758, 627926...",POINT (627924.092 6931866.123),"POLYGON ((627420.205 6931890.873, 627420.832 6...",0.0
4,209,627826.686572,6.931810e+06,627926.686572,6.931710e+06,124.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627923.628 6931809.758, 627926...",POINT (627925.667 6931782.783),"POLYGON ((627423.985 6931790.873, 627424.612 6...",0.0


### Import Wild Green Space GeoJson

In [155]:
#wild greenspace

wild=gpd.read_file("data/geojson/wild_greenspace.geojson")

In [156]:
#Change data type to float

wild=wild.astype({
    'grid_area':'float',
    'total_gp_area':'float',
    'tgp_area%':'float',
    'dgs_area':'float',
    'dgs%':'float',
    'qusai_area':'float',
    'qgs%':'float',
    'water_area':'float',
    'water%':'float',
    'wild_area':'float',
    'wild%':'float'
})

In [157]:
centriod_buffer(wild)
wild.head()

,id,left,top,right,bottom,grid_area,total_gp_area,tgp_area%,dgs_area,dgs%,qusai_area,qgs%,water_area,water%,wild_area,wild%,geometry,centriod,buffer
0,205,627826.686572,6.932210e+06,627926.686572,6.932110e+06,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627911.921 6932119.478, 627926...",POINT (627919.464 6932114.783),"POLYGON ((627412.278 6932100.593, 627412.832 6..."
1,206,627826.686572,6.932110e+06,627926.686572,6.932010e+06,1251.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627912.289 6932109.758, 627926...",POINT (627920.385 6932062.276),"POLYGON ((627412.646 6932090.873, 627413.273 6..."
2,207,627826.686572,6.932010e+06,627926.686572,6.931910e+06,873.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627916.069 6932009.758, 627926...",POINT (627922.254 6931963.367),"POLYGON ((627416.425 6931990.873, 627417.053 6..."
3,208,627826.686572,6.931910e+06,627926.686572,6.931810e+06,495.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627919.848 6931909.758, 627926...",POINT (627924.092 6931866.123),"POLYGON ((627420.205 6931890.873, 627420.832 6..."
4,209,627826.686572,6.931810e+06,627926.686572,6.931710e+06,124.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627923.628 6931809.758, 627926...",POINT (627925.667 6931782.783),"POLYGON ((627423.985 6931790.873, 627424.612 6..."


In [158]:
wild["green_percentage"] = green_percentage(wild, "wild_area")
wild.head()

,id,left,top,right,bottom,grid_area,total_gp_area,tgp_area%,dgs_area,dgs%,qusai_area,qgs%,water_area,water%,wild_area,wild%,geometry,centriod,buffer,green_percentage
0,205,627826.686572,6.932210e+06,627926.686572,6.932110e+06,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627911.921 6932119.478, 627926...",POINT (627919.464 6932114.783),"POLYGON ((627412.278 6932100.593, 627412.832 6...",8.959444
1,206,627826.686572,6.932110e+06,627926.686572,6.932010e+06,1251.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627912.289 6932109.758, 627926...",POINT (627920.385 6932062.276),"POLYGON ((627412.646 6932090.873, 627413.273 6...",12.194355
2,207,627826.686572,6.932010e+06,627926.686572,6.931910e+06,873.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627916.069 6932009.758, 627926...",POINT (627922.254 6931963.367),"POLYGON ((627416.425 6931990.873, 627417.053 6...",16.585612
3,208,627826.686572,6.931910e+06,627926.686572,6.931810e+06,495.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627919.848 6931909.758, 627926...",POINT (627924.092 6931866.123),"POLYGON ((627420.205 6931890.873, 627420.832 6...",18.438305
4,209,627826.686572,6.931810e+06,627926.686572,6.931710e+06,124.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627923.628 6931809.758, 627926...",POINT (627925.667 6931782.783),"POLYGON ((627423.985 6931790.873, 627424.612 6...",18.910401


### Import Water GeoJson

In [80]:
#water

water=gpd.read_file("data/geojson/water.geojson")

In [159]:
#Change data type to float

water=water.astype({
    'grid_area':'float',
    'total_gp_area':'float',
    'tgp_area%':'float',
    'dgs_area':'float',
    'dgs%':'float',
    'qusai_area':'float',
    'qgs%':'float',
    'water_area':'float',
    'water%':'float',
    'wild_area':'float',
    'wild%':'float'
})

In [160]:
centriod_buffer(water)
water.head()

,id,left,top,right,bottom,grid_area,total_gp_area,tgp_area%,dgs_area,dgs%,qusai_area,qgs%,water_area,water%,wild_area,wild%,geometry,centriod,buffer
0,205,627826.686572,6.932210e+06,627926.686572,6.932110e+06,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627911.921 6932119.478, 627926...",POINT (627919.464 6932114.783),"POLYGON ((627412.278 6932100.593, 627412.832 6..."
1,206,627826.686572,6.932110e+06,627926.686572,6.932010e+06,1251.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627912.289 6932109.758, 627926...",POINT (627920.385 6932062.276),"POLYGON ((627412.646 6932090.873, 627413.273 6..."
2,207,627826.686572,6.932010e+06,627926.686572,6.931910e+06,873.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627916.069 6932009.758, 627926...",POINT (627922.254 6931963.367),"POLYGON ((627416.425 6931990.873, 627417.053 6..."
3,208,627826.686572,6.931910e+06,627926.686572,6.931810e+06,495.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627919.848 6931909.758, 627926...",POINT (627924.092 6931866.123),"POLYGON ((627420.205 6931890.873, 627420.832 6..."
4,209,627826.686572,6.931810e+06,627926.686572,6.931710e+06,124.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627923.628 6931809.758, 627926...",POINT (627925.667 6931782.783),"POLYGON ((627423.985 6931790.873, 627424.612 6..."


In [161]:
water["green_percentage"] = green_percentage(water, "water_area")
water.head()

,id,left,top,right,bottom,grid_area,total_gp_area,tgp_area%,dgs_area,dgs%,qusai_area,qgs%,water_area,water%,wild_area,wild%,geometry,centriod,buffer,green_percentage
0,205,627826.686572,6.932210e+06,627926.686572,6.932110e+06,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627911.921 6932119.478, 627926...",POINT (627919.464 6932114.783),"POLYGON ((627412.278 6932100.593, 627412.832 6...",3.818492
1,206,627826.686572,6.932110e+06,627926.686572,6.932010e+06,1251.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627912.289 6932109.758, 627926...",POINT (627920.385 6932062.276),"POLYGON ((627412.646 6932090.873, 627413.273 6...",9.698737
2,207,627826.686572,6.932010e+06,627926.686572,6.931910e+06,873.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627916.069 6932009.758, 627926...",POINT (627922.254 6931963.367),"POLYGON ((627416.425 6931990.873, 627417.053 6...",11.017470
3,208,627826.686572,6.931910e+06,627926.686572,6.931810e+06,495.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627919.848 6931909.758, 627926...",POINT (627924.092 6931866.123),"POLYGON ((627420.205 6931890.873, 627420.832 6...",11.169989
4,209,627826.686572,6.931810e+06,627926.686572,6.931710e+06,124.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((627923.628 6931809.758, 627926...",POINT (627925.667 6931782.783),"POLYGON ((627423.985 6931790.873, 627424.612 6...",9.005937
